# Colab Runner (keep code in .py, run on Colab GPU)

This notebook is a **launcher**. Your project stays as `.py` + `pyproject.toml`.

## How to use (VS Code + Colab)
1) Open this `.ipynb` in VS Code.
2) Select a **Colab** kernel/runtime.
3) Edit `REPO_URL` once, run cells top→down.


In [ ]:
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())
print('Executable:', sys.executable)

# GPU check (safe if CPU runtime)
import subprocess
try:
    out = subprocess.check_output(['nvidia-smi'], stderr=subprocess.STDOUT, text=True)
    print(out[:2000])
except Exception as e:
    print('[info] nvidia-smi not available (CPU runtime or driver not exposed):', e)


In [ ]:
# --- EDIT THIS ONCE ---
REPO_URL = 'https://github.com/<YOUR_ID>/<YOUR_REPO>.git'
REPO_DIR = '<YOUR_REPO>'  # folder name after clone

# Optional: checkout a branch (e.g., PR branch)
BRANCH = ''  # e.g. 'chore/codex-ci' or leave empty for default

# Which project to run
PROJECT_PATH = 'ai-portfolio/01_nlp/nlp_sentiment_starter'

# Profile: if you have configs/colab.yaml use 'colab'. Otherwise keep 'local'.
PROFILE = 'colab'


In [ ]:
import os, pathlib, subprocess

def sh(cmd: str):
    print('>>', cmd)
    return subprocess.check_call(cmd, shell=True)

# Clone or update
if not pathlib.Path(REPO_DIR).exists():
    sh(f'git clone {REPO_URL} {REPO_DIR}')
os.chdir(REPO_DIR)
sh('git status --porcelain || true')
sh('git fetch --all --prune')
if BRANCH.strip():
    sh(f'git checkout {BRANCH}')
    sh('git pull')
print('Repo cwd:', os.getcwd())


In [ ]:
# Install uv inside Colab runtime (fast) and sync deps
import sys, subprocess, pathlib, os

subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'uv'])
subprocess.check_call(['uv', '--version'])

# Move into the project
os.chdir(PROJECT_PATH)
print('Project cwd:', os.getcwd())

# Decide profile fallback
cfg_colab = pathlib.Path('configs/colab.yaml')
if PROFILE == 'colab' and not cfg_colab.exists():
    print('[warn] configs/colab.yaml not found -> falling back to profile=local')
    PROFILE = 'local'

# Base sync (dev is optional)
subprocess.check_call(['uv', 'sync'])
subprocess.check_call(['uv', 'run', 'smoke', '--profile', PROFILE])


In [ ]:
# Run a full NLP demo (lightweight). Add extras when you need them.
import subprocess

# Optional extras:
# subprocess.check_call(['uv', 'sync', '--extra', 'dev'])
# subprocess.check_call(['uv', 'run', 'lint'])
# subprocess.check_call(['uv', 'run', 'test'])

# For real ML path (if your project supports it):
# subprocess.check_call(['uv', 'sync', '--extra', 'ml'])

subprocess.check_call(['uv', 'run', 'train', '--profile', PROFILE])
subprocess.check_call(['uv', 'run', 'eval', '--profile', PROFILE])
subprocess.check_call(['uv', 'run', 'predict', '--profile', PROFILE, '--text', 'this is surprisingly good'])


## Notes
- If you want **persistent outputs**, mount Google Drive and point your `configs/colab.yaml` paths into Drive.
- Colab's Python version can change by runtime; always verify with the first cell.
